In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
import pandas as pd

# Load dataset
dataset = pd.read_csv("NHANES_data_train.csv")
MI = dataset[dataset['MI'] == 1]
NoMI = dataset[dataset['MI'] == 2]

# Limit number of NoMI samples (145/3934 = .0368)
NoMI = NoMI.sample(frac=.0368)
dataset = pd.concat([MI, NoMI])

# Impute columns with nearest co-value
dataset_for_knn = dataset.drop(columns=['ParticipantID']) # ignore the ID column
imputer = KNNImputer(n_neighbors=1)
imputed_data = pd.DataFrame(imputer.fit_transform(dataset_for_knn), columns=dataset_for_knn.columns)

# Scale the data in the dataset, using values most relevent for prediction of MI
X = imputed_data[['Age', 'Diastolic', 'Systolic', 'Pulse', 'BMI', 'HDL', 'Trig', 'LDL', 'TCHOL', 'kidneys_eGFR', 'Diabetes', 'CurrentSmoker']]
Y = imputed_data['MI']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,stratify=Y, random_state=10)

# Find the best k value for the dataset
max_accuracy = 0
optimal_k = None
while max_accuracy < 0.8:
    k_range = range(1, 20, )
    scores = []
    for k in k_range:
        clf2 = Pipeline(
            steps=[("scaler", StandardScaler()), \
                   ("knn", KNeighborsClassifier(n_neighbors=k))]
        )
        clf2.fit(X_train, Y_train)
        scores.append(clf2.score(X_test, Y_test))
    
    max_accuracy = max(scores)
    if max_accuracy >= 0.8:
        optimal_k = k_range[scores.index(max_accuracy)]
        print(f" K value chosen: {optimal_k}")
    else:
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, random_state=None, test_size=.15)

# Train the model using the optimal k-value
clf = Pipeline(
    steps=[("scaler", StandardScaler()), 
           ("knn", KNeighborsClassifier(n_neighbors=optimal_k))]
)
clf.fit(X_train, Y_train)

# Make prediction
probabilities = clf.predict_proba(X_test)[:, 1]

# Get each sample's ID and probability, and write them to the output csv
dataset.reset_index(drop=True, inplace=True) # reset index
participant_ids = dataset.loc[X_test.index, 'ParticipantID']
output_df = pd.DataFrame({'ParticipantID': participant_ids, 'Probability_MI_Correct': probabilities})
output_df.to_csv('kNN_pred.csv', index=False)

# Merge the dataset with prediction set for accuracy evaluation, and send it to the csv file
orig_testing_data = pd.read_csv("NHANES_data_train.csv")
kNN_prediction = pd.read_csv("kNN_pred.csv")
merged_data = pd.merge(orig_testing_data, kNN_prediction, on='ParticipantID', how='inner')
kNN_true_MI_Label = merged_data[['ParticipantID', 'MI']]
kNN_true_MI_Label.to_csv('kNN_true_MI_Label.csv', index=False)


 K value chosen: 8
